<a href="https://colab.research.google.com/github/hfoffani/HF-fastai/blob/main/nb11_iula_build_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Procesar Corpus

In [3]:
keys = {
    'a':'ambiente',
    'm':'medicina',
    'i':'tecnología',
    'e':'economía',
    'd':'legal',
}

def sentences(corpusroot: str) -> pd.DataFrame:
    listfiles = sorted(glob.glob(corpusroot + '/*/*-plain.txt'))
    corpus = []
    lastcl = ''
    for fname in listfiles:
        cl = os.path.basename(fname)[0]
        if cl != lastcl:
            lastcl = cl
            docnum = 0
        docnum += 1
        with open(fname) as f:
            lines = f.readlines()
            for l in lines:
                l = l.strip()
                if len(l) > 1:
                    corpus.append( {
                        'label':keys[cl], 'docnum': docnum, 'sentence':l } )
    return pd.DataFrame(corpus)

def split_df(corpus_df:pd.DataFrame, split:float=.20) -> pd.DataFrame:
    corpus_df['is_valid'] = (corpus_df['docnum'] % (1./split) == 0)
    return corpus_df


In [5]:
co_df = sentences('/content/drive/MyDrive/IULAcorpus/ES')
co_df = split_df(co_df, .20)

co_df.groupby(['label','is_valid']).nunique()

docnum  sentence
label      is_valid                  
ambiente   False          9      2556
           True           2      1647
economía   False          8      4588
           True           1       205
legal      False          1       483
medicina   False         68     21993
           True          17      4143
tecnología False         17     11985
           True           4      2861

In [6]:
co_df.drop(co_df[co_df.label == 'legal'].index, inplace=True)
co_df.groupby(['label','is_valid']).nunique()

docnum  sentence
label      is_valid                  
ambiente   False          9      2556
           True           2      1647
economía   False          8      4588
           True           1       205
medicina   False         68     21993
           True          17      4143
tecnología False         17     11985
           True           4      2861

In [7]:
co_df.to_csv('/content/drive/MyDrive/iula-es-noleaky.csv', index=False)